In [1]:
import pandas as pd
import numpy as np
import joblib
from tqdm import tqdm

In [2]:
tqdm.pandas()

In [3]:
df = joblib.load("../Dump/df.pkl")
# df = pd.read_excel("../Data/ar-2010-2014-xlsb.xlsb", engine="pyxlsb")
# df

In [4]:
pcts = []
statuses = []

for cols in tqdm(df.columns):
    pct = df[cols].isnull().sum() / df.shape[0]
    pcts.append(pct)
    
    if pct < 0.1:
        status = "NA values handling"
    else:
        status = "Throw away"
        
    statuses.append(status)
    
#     print(f'Columns: {cols}\n null-percentages:{pct*100:.2f}%')

nulls = pd.DataFrame({"Col": df.columns, "pct": pcts, "status": statuses})
nulls

100%|██████████| 95/95 [00:00<00:00, 166.94it/s]


,Col,pct,status
0,Patient Age at Treatment,0.000000,NA values handling
1,Date patient started trying to become pregnant...,0.963590,Throw away
2,"Total Number of Previous cycles, Both IVF and DI",0.000000,NA values handling
3,"Total Number of Previous treatments, Both IVF ...",0.000000,NA values handling
4,Total Number of Previous IVF cycles,0.000000,NA values handling
...,...,...,...
90,Heart Four Birth Outcome,0.999979,Throw away
91,Heart Four Birth Weight,0.999994,Throw away
92,Heart Four Sex,0.999994,Throw away
93,Heart Four Delivery Date,0.999994,Throw away


In [6]:
df_must_handle = df[nulls[nulls["status"] == "NA values handling"]["Col"].values]
df_must_handle['Live Birth Occurrence'] = df['Live Birth Occurrence']
df_must_handle.head()

C:\Users\M.FAHRURY\AppData\Local\Temp\ipykernel_20468\3611178038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_must_handle['Live Birth Occurrence'] = df['Live Birth Occurrence']


,Patient Age at Treatment,"Total Number of Previous cycles, Both IVF and DI","Total Number of Previous treatments, Both IVF and DI at clinic",Total Number of Previous IVF cycles,Total Number of Previous DI cycles,"Total number of previous pregnancies, Both IVF and DI",Total number of IVF pregnancies,Total number of DI pregnancies,Total number of live births - conceived through IVF or DI,Total number of live births - conceived through IVF,...,Embryos from Eggs Micro-injected,Total Embryos Thawed,Embryos Transfered,Embryos Transfered from Eggs Micro-injected,Embryos Stored For Use By Patient,Embryos (from Eggs Micro-injected) Stored For Use By Patient,Year of Treatment,Number of Live Births,Number of foetal sacs with fetal pulsation,Live Birth Occurrence
0,18 - 34,1,1,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2010,2,2,1.0
1,35-37,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2013,0,0,NaN
2,18 - 34,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2012,2,2,1.0
3,38-39,1,1,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2014,0,0,NaN
4,35-37,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,2010,0,0,NaN


In [8]:
# inpect NaN value in target variable ('Live Birth Occurrence')
print(f'Null value percentages: {(df_must_handle["Live Birth Occurrence"].isnull().sum()/df.shape[0])*100:.2f}%')
print(f'Unique values inside are: {df_must_handle["Live Birth Occurrence"].unique()}')


Null value percentages: 75.93%
Unique values inside are: [ 1. nan]


maybe all zero or negative class recorded as NaN

let's convert NaN to zero class

In [9]:
df_must_handle[df_must_handle['Live Birth Occurrence'].isnull() == True] = 0


C:\Users\M.FAHRURY\AppData\Local\Temp\ipykernel_20468\3393358916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_must_handle[df_must_handle['Live Birth Occurrence'].isnull() == True] = 0


In [10]:
df_must_handle['Live Birth Occurrence'].unique()

array([1., 0.])

In [11]:
joblib.dump(df_must_handle, "../Dump/df_must_handle.pkl")

['../Dump/df_must_handle.pkl']

now all the NaN have been converted to 0

### Data Splitting

In [42]:
X = df_must_handle.drop(['Live Birth Occurrence'], axis=1)
y = df_must_handle['Live Birth Occurrence']

print(f'X.shape: {X.shape}')
print(f'y.shape: {y.shape}')

X.shape: (337109, 57)
y.shape: (337109,)


In [44]:
from sklearn.model_selection import train_test_split
# train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
# test val
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# print shape
print(f'X_train, y_train: {X_train.shape},{y_train.shape}')
print(f'X_valid, y_valid: {X_valid.shape},{y_valid.shape}')
print(f'X_test, y_test: {X_test.shape},{y_test.shape}')

X_train, y_train: (202265, 57),(202265,)
X_valid, y_valid: (67422, 57),(67422,)
X_test, y_test: (67422, 57),(67422,)


In [70]:
# check class balance
before_splitting = {
    "0": f'{(df_must_handle["Live Birth Occurrence"].value_counts()[0]/df_must_handle.shape[0])*100:.2f}%',
    "1": f'{(df_must_handle["Live Birth Occurrence"].value_counts()[1]/df_must_handle.shape[0])*100:.2f}%'
}

after_splitting = {
    "0_train": f'{(y_train.value_counts()[0]/y_train.shape[0])*100:.2f}%',
    "1_train": f'{(y_train.value_counts()[1]/y_train.shape[0])*100:.2f}%',

    "0_valid": f'{(y_valid.value_counts()[0]/y_valid.shape[0])*100:.2f}%',
    "1_valid": f'{(y_valid.value_counts()[1]/y_valid.shape[0])*100:.2f}%',
    
    "0_test": f'{(y_test.value_counts()[0]/y_test.shape[0])*100:.2f}%',
    "1_test": f'{(y_test.value_counts()[1]/y_test.shape[0])*100:.2f}%',
}

print(f'Before splitting: {before_splitting}')
print(f'After splitting: {after_splitting}')

Before splitting: {'0': '75.93%', '1': '24.07%'}
After splitting: {'0_train': '75.93%', '1_train': '24.07%', '0_valid': '75.93%', '1_valid': '24.07%', '0_test': '75.94%', '1_test': '24.06%'}


Data splitting do the split in stratify manner as well as we hope

In [72]:
# dump the train, valid, test
joblib.dump(X_train, "../Dump/x_train.pkl")
joblib.dump(y_train, "../Dump/y_train.pkl")
joblib.dump(X_valid, "../Dump/x_valid.pkl")
joblib.dump(y_valid, "../Dump/y_valid.pkl")
joblib.dump(X_test, "../Dump/x_test.pkl")
joblib.dump(y_test, "../Dump/y_test.pkl")

['../Dump/y_test.pkl']

In [12]:
# load the splitted data
x_train = joblib.load("../Dump/x_train.pkl")
y_train = joblib.load("../Dump/y_train.pkl")
x_valid = joblib.load("../Dump/x_valid.pkl")
y_valid = joblib.load("../Dump/y_valid.pkl")
x_test = joblib.load("../Dump/x_test.pkl")
y_test = joblib.load("../Dump/y_test.pkl")

### Pre-processing